In [1]:
%reload_ext autoreload
%autoreload 2

# Load the data

In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [15]:
from Scratch import metadata_loader
import numpy as np
from tqdm import tqdm
from time import sleep

DEVICE = "ibm_sherbrooke"
LOGICAL = str(0)
XBASIS = True
ROUNDS = 50

state = "X" if XBASIS else "Z"
state += LOGICAL
print(f"state = {state}")


# Load the metadata
while True:
    try:
        md = metadata_loader(True, True)
        break
    except:
        sleep(5)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md[md["descr"] == 'subset RepCodes']
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
md = md[md["logical"] == LOGICAL]
md = md[md["xbasis"] == XBASIS]
md = md[md["rounds"] == ROUNDS]

# md = md[30:31]
print("shape:", md.shape)
# md[18:]
# md

state = X0


shape: (40, 18)


# Group job ids by closest calibration date

In [16]:
from datetime import datetime
from Scratch import find_closest_calib_jobs
import pandas as pd

DOUBLE_MSMT = False # to get also 03-16

jobs_by_calibration_date = {}
for index, row in md.iterrows():
    job_id = row['job_id']

    while True:
        try:
            _, _, calib_creation_date = find_closest_calib_jobs(tobecalib_job=job_id, verbose=False, double_msmt=DOUBLE_MSMT)
            break
        except:
            sleep(5)

    if calib_creation_date not in jobs_by_calibration_date.keys():
        jobs_by_calibration_date[calib_creation_date] = [job_id]
    else:
        jobs_by_calibration_date[calib_creation_date].append(job_id)

# Takes 15s

In [17]:
print(jobs_by_calibration_date)
print()
print(f"num of calibrations: {len(jobs_by_calibration_date)}")
print(f"num of jobs per calibration: {([len(jobs) for jobs in jobs_by_calibration_date.values()])}")

{datetime.datetime(2024, 3, 23, 13, 12, 1, 375000, tzinfo=datetime.timezone.utc): ['cqzda58czq6g0081he8g', 'cqzda38dvs8g008j69vg', 'cqzda1g8gdp0008fvt4g', 'cqzd9zqdvs8g008j69t0', 'cqzd9xzs9z7g008dpajg', 'cqzd9wfczq6g0081he80', 'cqzd9tfktf3g00883pdg', 'cqzd9rf8gdp0008fvt30', 'cqzd9pys9z7g008dpahg', 'cqzd9mpdvs8g008j69sg', 'cqzd9k68gdp0008fvt2g', 'cqzd9he8gdp0008fvt20'], datetime.datetime(2024, 3, 23, 13, 8, 53, 174000, tzinfo=datetime.timezone.utc): ['cqzd9e5k5z70008j0880', 'cqzd9addvs8g008j69s0', 'cqzd98d8gdp0008fvt1g', 'cqzd96ck5z70008j087g', 'cqzd92c8gdp0008fvt0g', 'cqzd904ktf3g00883pd0', 'cqzd8y3dvs8g008j69r0', 'cqzd8wbs9z7g008dpag0'], datetime.datetime(2024, 3, 16, 10, 33, 33, 406000, tzinfo=datetime.timezone.utc): ['cqtqaxv4x0mg008aj1w0', 'cqtqawbtxzj0008y0260', 'cqtqatvtxzj0008y025g', 'cqtqasb4x0mg008aj1vg', 'cqtqaqj88ev0008130eg', 'cqtqapapkcdg008e3dhg', 'cqtqama36d60008hzs70', 'cqtqajj4x0mg008aj1v0', 'cqtqah236d60008hzs6g', 'cqtqafstxzj0008y0250'], datetime.datetime(2024, 3, 16

# Decode data

In [18]:
d = md["distance"].values[0]

distances = np.arange(7, d+1, 4)
distances = distances[::-1]

distances

array([51, 47, 43, 39, 35, 31, 27, 23, 19, 15, 11,  7])

In [19]:
import json
import os
import time

import pymatching
import stim

from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, get_repcode_IQ_map
from soft_info import RepetitionCodeStimCircuit, inv_qubit_mapping, gaussianIQConvertor
from soft_info import get_cols_to_keep, generate_subsets_with_center, get_subsample_layout
from Scratch import load_calibration_memory
import cpp_soft_info as csi

file_name  = f'./results_night1/{DEVICE}_{state}_{ROUNDS}.json'

HANDLE_OUTLIERS = True

rel_error = 1
_RESETS = False
nb_intervals = -1

# KDE BANDWIDTHS
lin = [0.6, 1.2, 1]
num_points = 20
# lin = [0.1, 0.7, 1]
# num_points = 7
bandwidths = np.linspace(lin[0], lin[1], lin[2])


for calib_date in jobs_by_calibration_date.keys():
    # get the noise dict of that date
    noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

    # get the KDE of that date

    while True:
        try:
            all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                        other_date=calib_date, post_process=True,
                                                                        double_msmt=False)
            break
        except:
            sleep(5)

    kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

    while True:
        try:
            all_memories_PS, gmm_dict_PS, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                            other_date=calib_date, post_process=True,
                                                                            double_msmt=True)
            break
        except:
            sleep(5)
    kde_dict_PS = csi.get_KDEs(all_memories_PS, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

    # Get the mean msmt errors
    p_soft_mean = 0
    p_hard_mean = 0
    for key, value in msmt_err_dict_PS.items():
        p_soft_mean += value['p_soft']
        p_hard_mean += value['p_hard']
    p_soft_mean /= len(msmt_err_dict_PS)
    p_hard_mean /= len(msmt_err_dict_PS)
    print(msmt_err_dict_PS, p_soft_mean, p_hard_mean)

    for i in range(127):
        print(f"qubit {i}: {kde_dict[i].bestBandwidth}", end=' ')
    print("\nPS", end=' ')
    for i in range(127):
        print(f"qubit {i}: {kde_dict_PS[i].bestBandwidth}", end=' ')

    # Retrieve the memories
    memories = []
    for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Retrieving jobs of {calib_date} calibration"):
        d = md[md["job_id"] == job_id]["distance"].values[0] 
        T = md[md["job_id"] == job_id]["rounds"].values[0] # Should be 10

        # Get the job
        job = provider.retrieve_job(job_id)
        memory = job.result().get_memory()
        memories.append(memory)

    # Stack the memories vertically
    big_memory = np.vstack(memories)
    NB_SHOTS = big_memory.shape[0]

    # Get the layout of the last job (same as previous)
    layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
    link_qubits = list(layout_des['link_qubit'].values())
    code_qubits = list(layout_des['code_qubit'].values())

    # Get the pSoft and countMat matrices
    big_layout = link_qubits + code_qubits
    inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))

    print(f"Starting to get pSoft and countMat at {datetime.now()}")
    pSoft, countMat = csi.iqConvertor(big_memory, inverted_q_map, kde_dict, rel_error, -1,
                                                handleOutliers = HANDLE_OUTLIERS)
    
    print(f"Starting to get pSoft_PS and countMat_PS at {datetime.now()}")
    pSoft_PS, countMat_PS = csi.iqConvertor(big_memory, inverted_q_map, kde_dict_PS, rel_error, -1,
                                                      handleOutliers = HANDLE_OUTLIERS)
    
    print(f"Starting to get pSoftG at {datetime.now()}")
    countMatG, pSoftG = gaussianIQConvertor(big_memory, inverted_q_map, gmm_dict)


    # Subsample decoding
    for D_NEW in tqdm(distances):
        subsets = generate_subsets_with_center(d, D_NEW)

        err_s_G_mean = []
        err_s_G_indiv = []
        err_h_G_indiv = []
        err_h_G_mean = []
        err_s_K_mean = []
        err_s_K_indiv = []
        err_h_K_indiv = []
        err_h_K_mean = []
        err_s_KPS_mean = []
        err_s_KPS_indiv = []
        err_h_KPS_indiv = []
        err_h_KPS_mean = []
        err_h_G_mean_mean = []
        err_h_K_mean_mean = []
        err_h_KPS_mean_mean = []
        for subset in (subsets):             
            cols_to_keep = get_cols_to_keep(subset, T, d)

            # Get the subset of pSoft and countMat
            pSoft_sub = pSoft[:, cols_to_keep]
            pSoft_PS_sub = pSoft_PS[:, cols_to_keep]
            pSoftG_sub = pSoftG[:, cols_to_keep]
            countMat_sub = countMat[:, cols_to_keep]
            countMat_PS_sub = countMat_PS[:, cols_to_keep]
            countMatG_sub = countMatG[:, cols_to_keep]

            # Means for hard decoding
            pSoft_mean = np.mean(pSoft_sub)
            pSoft_PS_mean = np.mean(pSoft_PS_sub)
            pSoftG_mean = np.mean(pSoftG_sub)

            # Get the layout
            layout = get_subsample_layout(subset, link_qubits, code_qubits)
            qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=T)

            # Get the noise avgs
            avgs = get_avgs_from_dict(noise_dict, layout)
            noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
            readout = avgs["readout"]
            noise_list += [(p_hard_mean+p_soft_mean), p_hard_mean, p_soft_mean]
            #[twog_err, sglg_err, t1_err, t2_err, readout_err, hard_err, soft_err]

            # Stim models
            subsampling = (D_NEW != d)

            code_indiv_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=layout,
                                                            msmt_err_dict=msmt_err_dict_PS)
            model_indiv_for_soft = code_indiv_for_soft.circuits[LOGICAL].detector_error_model()

            code_indiv_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=layout,
                                                            msmt_err_dict=msmt_err_dict_PS)
            model_indiv_for_hard = code_indiv_for_hard.circuits[LOGICAL].detector_error_model()

            code_mean_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_soft = code_mean_for_soft.circuits[LOGICAL].detector_error_model()

            code_mean_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_hard = code_mean_for_hard.circuits[LOGICAL].detector_error_model()
            

            # Decoding

            ### Gaussian
            res_s_G_mean = csi.decodeConvertorAll(model_mean_for_soft, countMatG_sub, pSoftG_sub, T,
                                                  int(LOGICAL), _RESETS, decode_hard=False)
            res_s_G_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_G_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_G_mean = csi.decodeConvertorAll(model_mean_for_hard, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoftG_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_G_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            ### KDE
            res_s_K_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_s_K_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_K_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_K_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoft_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                        subsampling=subsampling, no_fin_soft=False, layout=None,
                                                        msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_K_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
            
            ### KDE PS
            res_s_KPS_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_s_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_KPS_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoft_PS_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                        subsampling=subsampling, no_fin_soft=False, layout=None,
                                                        msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_KPS_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
            
            # Append the results
            err_s_G_mean.append(res_s_G_mean.num_errors)
            err_s_G_indiv.append(res_s_G_indiv.num_errors)
            err_h_G_indiv.append(res_h_G_indiv.num_errors)
            err_h_G_mean.append(res_h_G_mean.num_errors)
            err_s_K_mean.append(res_s_K_mean.num_errors)
            err_s_K_indiv.append(res_s_K_indiv.num_errors)
            err_h_K_indiv.append(res_h_K_indiv.num_errors)
            err_h_K_mean.append(res_h_K_mean.num_errors)
            err_s_KPS_mean.append(res_s_KPS_mean.num_errors)
            err_s_KPS_indiv.append(res_s_KPS_indiv.num_errors)
            err_h_KPS_indiv.append(res_h_KPS_indiv.num_errors)
            err_h_KPS_mean.append(res_h_KPS_mean.num_errors)

            err_h_G_mean_mean.append(res_h_G_mean_mean.num_errors)
            err_h_K_mean_mean.append(res_h_K_mean_mean.num_errors)
            err_h_KPS_mean_mean.append(res_h_KPS_mean_mean.num_errors)


        # Save the results
        json_Gaussian = {
            "decoding": "gaussian",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_G_mean,
                "soft_indiv": err_s_G_indiv,
                "hard_indiv": err_h_G_indiv,
                "hard_mean": err_h_G_mean,
                "hard_mean_mean": err_h_G_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_G_mean),
                "soft_indiv": np.mean(err_s_G_indiv),
                "hard_indiv": np.mean(err_h_G_indiv),
                "hard_mean": np.mean(err_h_G_mean),
                "hard_mean_mean": np.mean(err_h_G_mean_mean),
            },      
            "additional_info": {
                "calib_date": str(calib_date),
                "noise_list": noise_list,
                "calib_readout": readout,
                "p_soft_mean": p_soft_mean,
                "p_hard_mean": p_hard_mean,
                "p_softG_mean": pSoftG_mean,
            },    
        }

        json_KDE = {
            "decoding": "kde",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_K_mean,
                "soft_indiv": err_s_K_indiv,
                "hard_indiv": err_h_K_indiv,
                "hard_mean": err_h_K_mean,
                "hard_mean_mean": err_h_K_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_K_mean),
                "soft_indiv": np.mean(err_s_K_indiv),
                "hard_indiv": np.mean(err_h_K_indiv),
                "hard_mean": np.mean(err_h_K_mean),
                "hard_mean_mean": np.mean(err_h_K_mean_mean),
            },   
            "additional_info": {
                "rel_error": rel_error,
                "bandwidth_linspace": lin,
                "num_points_bandwidths": num_points,
                "pSoft_mean": pSoft_mean,
            },        
        }

        json_KDE_PS = {
            "decoding": "kde_PS",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_KPS_mean,
                "soft_indiv": err_s_KPS_indiv,
                "hard_indiv": err_h_KPS_indiv,
                "hard_mean": err_h_KPS_mean,
                "hard_mean_mean": err_h_KPS_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_KPS_mean),
                "soft_indiv": np.mean(err_s_KPS_indiv),
                "hard_indiv": np.mean(err_h_KPS_indiv),
                "hard_mean": np.mean(err_h_KPS_mean),
                "hard_mean_mean": np.mean(err_h_KPS_mean_mean),
                "pSoft_PS_mean": pSoft_PS_mean,
            },   
        }

        if not os.path.exists(file_name):
            data = {}
        else:
            with open(file_name, "r") as f:
                data = json.load(f)

        if job_id not in data.keys():
            data[job_id] = [{'totshots': NB_SHOTS}, json_Gaussian, json_KDE, json_KDE_PS]
        else:
            data[job_id].append(json_Gaussian)
            data[job_id].append(json_KDE)
            data[job_id].append(json_KDE_PS)
        
        with open(file_name, "w") as f:
            json.dump(data, f, indent=4)

08:11:15 Warning: Z0 decoding. Negative T2 error -0.08 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
08:11:15 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
08:11:15 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 111, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 13:11:56.380893+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 13:56:07.037111+00:00.
{0: {'p_hard': 0.003841757683515367, 'p_soft': 0.000127000254000508}, 1: {'p_hard': 0.005492760985521971, 'p_soft': 0.004572

Retrieving jobs of 2024-03-23 13:12:01.375000+00:00 calibration: 100%|██████████| 12/12 [00:30<00:00,  2.57s/it]


Starting to get pSoft and countMat at 2024-04-01 08:12:57.456076
Starting to get pSoft_PS and countMat_PS at 2024-04-01 08:17:16.593885
Starting to get pSoftG at 2024-04-01 08:20:05.221111


100%|██████████| 12/12 [18:12<00:00, 91.03s/it]


08:38:19 Warning: Z0 decoding. Negative T2 error -0.08 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
08:38:19 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
08:38:19 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 111, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 13:11:56.380893+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 13:40:36.409502+00:00.
{0: {'p_hard': 0.003397256794513589, 'p_soft': 6.3500127000254e-05}, 1: {'p_hard': 0.005143510287020574, 'p_soft': 0.0049530

...

100%|██████████| 12/12 [16:20<00:00, 81.67s/it]


09:39:06 Warning: Z0 decoding. Negative T2 error -0.06 % for qubit 123, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-16 10:11:14.156336+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 09:05:45.817173+00:00.
{0: {'p_hard': 0.004667259334518669, 'p_soft': 6.3500127000254e-05}, 1: {'p_hard': 0.012192024384048768, 'p_soft': 0.004318008636017272}, 2: {'p_hard': 0.014319278638557276, 'p_soft': 0.011271272542545086}, 3: {'p_hard': 0.010922021844043688, 'p_soft': 0.001079502159004318}, 4: {'p_hard': 0.007112014224028448, 'p_soft': 0.0024765049530099063}, 5: {'p_hard': 0.010922021844043688, 'p_soft': 0.03321056642113284}, 6: {'p_hard': 0.019335788671577342, 'p_soft': 0.02136779273558547}, 7: {'p_hard': 0.006127762255524511, 'p_soft': 0.0015875031750063502}, 8: {'p_hard': 0.004921

Retrieving jobs of 2024-03-16 10:31:16.396000+00:00 calibration: 100%|██████████| 10/10 [00:26<00:00,  2.65s/it]


Starting to get pSoft and countMat at 2024-04-01 09:40:37.019566
Starting to get pSoft_PS and countMat_PS at 2024-04-01 09:44:43.329250
Starting to get pSoftG at 2024-04-01 09:47:16.596316


100%|██████████| 12/12 [16:22<00:00, 81.85s/it]


# ----------------------------------------------------------

# ----------------------------------------------------------

# ----------------------------------------------------------

# ----------------------------------------------------------

# ----------------------------------------------------------

In [ ]:
from Scratch import metadata_loader
import numpy as np
from tqdm import tqdm
from time import sleep

DEVICE = "ibm_sherbrooke"
LOGICAL = str(0)
XBASIS = False
ROUNDS = 30

state = "X" if XBASIS else "Z"
state += LOGICAL
print(f"state = {state}")


# Load the metadata
while True:
    try:
        md = metadata_loader(True, True)
        break
    except:
        sleep(5)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md[md["descr"] == 'subset RepCodes']
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
md = md[md["logical"] == LOGICAL]
md = md[md["xbasis"] == XBASIS]
md = md[md["rounds"] == ROUNDS]

# md = md[30:31]
print("shape:", md.shape)
# md[18:]
# md

state = Z0
shape: (39, 18)


# Group job ids by closest calibration date

In [ ]:
from datetime import datetime
from Scratch import find_closest_calib_jobs
import pandas as pd

DOUBLE_MSMT = False # to get also 03-16

jobs_by_calibration_date = {}
for index, row in md.iterrows():
    job_id = row['job_id']

    while True:
        try:
            _, _, calib_creation_date = find_closest_calib_jobs(tobecalib_job=job_id, verbose=False, double_msmt=DOUBLE_MSMT)
            break
        except:
            sleep(5)

    if calib_creation_date not in jobs_by_calibration_date.keys():
        jobs_by_calibration_date[calib_creation_date] = [job_id]
    else:
        jobs_by_calibration_date[calib_creation_date].append(job_id)

# Takes 15s

In [ ]:
print(jobs_by_calibration_date)
print()
print(f"num of calibrations: {len(jobs_by_calibration_date)}")
print(f"num of jobs per calibration: {([len(jobs) for jobs in jobs_by_calibration_date.values()])}")

{datetime.datetime(2024, 3, 23, 10, 49, 33, 412000, tzinfo=datetime.timezone.utc): ['cqzb7dds9z7g008dp66g', 'cqzb7cds9z7g008dp660', 'cqzb7bddvs8g008j66kg', 'cqzb7a5k5z70008j054g', 'cqzb7958gdp0008fvnj0', 'cqzb785s9z7g008dp650', 'cqzb774ktf3g00883jrg', 'cqzb764k5z70008j0540', 'cqzb754k5z70008j053g'], datetime.datetime(2024, 3, 23, 10, 48, 2, 487000, tzinfo=datetime.timezone.utc): ['cqzb744ktf3g00883jqg', 'cqzb734ktf3g00883jq0', 'cqzb724ktf3g00883jpg', 'cqzb71cczq6g0081h9yg', 'cqzb704ktf3g00883jp0', 'cqzb6yvczq6g0081h9y0', 'cqzb6xv8gdp0008fvngg', 'cqzb6wkktf3g00883jng', 'cqzb6vk8gdp0008fvng0', 'cqzb6tkk5z70008j0530'], datetime.datetime(2024, 3, 15, 22, 22, 45, 907000, tzinfo=datetime.timezone.utc): ['cqtcmchtxzj0008xz2dg', 'cqtcmb9txzj0008xz2d0', 'cqtcma988ev000811y6g', 'cqtcm9936d60008hyr20', 'cqtcm8988ev000811y60', 'cqtcm7088ev000811y5g', 'cqtcm6088ev000811y50', 'cqtcm50ch2mg008qvbjg', 'cqtcm409nfw0008hrvmg', 'cqtcm2r36d60008hyr1g'], datetime.datetime(2024, 3, 15, 22, 21, 14, 43000, tz

# Decode data

In [ ]:
d = md["distance"].values[0]

distances = np.arange(7, d+1, 4)
distances = distances[::-1]

distances

array([51, 47, 43, 39, 35, 31, 27, 23, 19, 15, 11,  7])

In [ ]:
import json
import os
import time

import pymatching
import stim

from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, get_repcode_IQ_map
from soft_info import RepetitionCodeStimCircuit, inv_qubit_mapping, gaussianIQConvertor
from soft_info import get_cols_to_keep, generate_subsets_with_center, get_subsample_layout
from Scratch import load_calibration_memory
import cpp_soft_info as csi

file_name  = f'./results_night1/{DEVICE}_{state}_{ROUNDS}.json'

HANDLE_OUTLIERS = True

rel_error = 1
_RESETS = False
nb_intervals = -1

# KDE BANDWIDTHS
lin = [0.6, 1.2, 1]
num_points = 20
# lin = [0.1, 0.7, 1]
# num_points = 7
bandwidths = np.linspace(lin[0], lin[1], lin[2])


for calib_date in jobs_by_calibration_date.keys():
    # get the noise dict of that date
    noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

    # get the KDE of that date

    while True:
        try:
            all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                        other_date=calib_date, post_process=True,
                                                                        double_msmt=False)
            break
        except:
            sleep(5)

    kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

    while True:
        try:
            all_memories_PS, gmm_dict_PS, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                            other_date=calib_date, post_process=True,
                                                                            double_msmt=True)
            break
        except:
            sleep(5)
    kde_dict_PS = csi.get_KDEs(all_memories_PS, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

    # Get the mean msmt errors
    p_soft_mean = 0
    p_hard_mean = 0
    for key, value in msmt_err_dict_PS.items():
        p_soft_mean += value['p_soft']
        p_hard_mean += value['p_hard']
    p_soft_mean /= len(msmt_err_dict_PS)
    p_hard_mean /= len(msmt_err_dict_PS)
    print(msmt_err_dict_PS, p_soft_mean, p_hard_mean)

    for i in range(127):
        print(f"qubit {i}: {kde_dict[i].bestBandwidth}", end=' ')
    print("\nPS", end=' ')
    for i in range(127):
        print(f"qubit {i}: {kde_dict_PS[i].bestBandwidth}", end=' ')

    # Retrieve the memories
    memories = []
    for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Retrieving jobs of {calib_date} calibration"):
        d = md[md["job_id"] == job_id]["distance"].values[0] 
        T = md[md["job_id"] == job_id]["rounds"].values[0] # Should be 10

        # Get the job
        job = provider.retrieve_job(job_id)
        memory = job.result().get_memory()
        memories.append(memory)

    # Stack the memories vertically
    big_memory = np.vstack(memories)
    NB_SHOTS = big_memory.shape[0]

    # Get the layout of the last job (same as previous)
    layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
    link_qubits = list(layout_des['link_qubit'].values())
    code_qubits = list(layout_des['code_qubit'].values())

    # Get the pSoft and countMat matrices
    big_layout = link_qubits + code_qubits
    inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))

    print(f"Starting to get pSoft and countMat at {datetime.now()}")
    pSoft, countMat = csi.iqConvertor(big_memory, inverted_q_map, kde_dict, rel_error, -1,
                                                handleOutliers = HANDLE_OUTLIERS)
    
    print(f"Starting to get pSoft_PS and countMat_PS at {datetime.now()}")
    pSoft_PS, countMat_PS = csi.iqConvertor(big_memory, inverted_q_map, kde_dict_PS, rel_error, -1,
                                                      handleOutliers = HANDLE_OUTLIERS)
    
    print(f"Starting to get pSoftG at {datetime.now()}")
    countMatG, pSoftG = gaussianIQConvertor(memory, inverted_q_map, gmm_dict)


    # Subsample decoding
    for D_NEW in tqdm(distances):
        subsets = generate_subsets_with_center(d, D_NEW)

        err_s_G_mean = []
        err_s_G_indiv = []
        err_h_G_indiv = []
        err_h_G_mean = []
        err_s_K_mean = []
        err_s_K_indiv = []
        err_h_K_indiv = []
        err_h_K_mean = []
        err_s_KPS_mean = []
        err_s_KPS_indiv = []
        err_h_KPS_indiv = []
        err_h_KPS_mean = []
        err_h_G_mean_mean = []
        err_h_K_mean_mean = []
        err_h_KPS_mean_mean = []
        for subset in (subsets):             
            cols_to_keep = get_cols_to_keep(subset, T, d)

            # Get the subset of pSoft and countMat
            pSoft_sub = pSoft[:, cols_to_keep]
            pSoft_PS_sub = pSoft_PS[:, cols_to_keep]
            pSoftG_sub = pSoftG[:, cols_to_keep]
            countMat_sub = countMat[:, cols_to_keep]
            countMat_PS_sub = countMat_PS[:, cols_to_keep]
            countMatG_sub = countMatG[:, cols_to_keep]

            # Means for hard decoding
            pSoft_mean = np.mean(pSoft_sub)
            pSoft_PS_mean = np.mean(pSoft_PS_sub)
            pSoftG_mean = np.mean(pSoftG_sub)

            # Get the layout
            layout = get_subsample_layout(subset, link_qubits, code_qubits)
            qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=T)

            # Get the noise avgs
            avgs = get_avgs_from_dict(noise_dict, layout)
            noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
            readout = avgs["readout"]
            noise_list += [(p_hard_mean+p_soft_mean), p_hard_mean, p_soft_mean]
            #[twog_err, sglg_err, t1_err, t2_err, readout_err, hard_err, soft_err]

            # Stim models
            subsampling = (D_NEW != d)

            code_indiv_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=layout,
                                                            msmt_err_dict=msmt_err_dict_PS)
            model_indiv_for_soft = code_indiv_for_soft.circuits[LOGICAL].detector_error_model()

            code_indiv_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=layout,
                                                            msmt_err_dict=msmt_err_dict_PS)
            model_indiv_for_hard = code_indiv_for_hard.circuits[LOGICAL].detector_error_model()

            code_mean_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_soft = code_mean_for_soft.circuits[LOGICAL].detector_error_model()

            code_mean_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_hard = code_mean_for_hard.circuits[LOGICAL].detector_error_model()
            

            # Decoding

            ### Gaussian
            res_s_G_mean = csi.decodeConvertorAll(model_mean_for_soft, countMatG_sub, pSoftG_sub, T,
                                                  int(LOGICAL), _RESETS, decode_hard=False)
            res_s_G_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_G_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_G_mean = csi.decodeConvertorAll(model_mean_for_hard, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoftG_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_G_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            ### KDE
            res_s_K_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_s_K_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_K_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_K_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoft_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                        subsampling=subsampling, no_fin_soft=False, layout=None,
                                                        msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_K_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
            
            ### KDE PS
            res_s_KPS_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_s_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_KPS_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoft_PS_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                        subsampling=subsampling, no_fin_soft=False, layout=None,
                                                        msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_KPS_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
            
            # Append the results
            err_s_G_mean.append(res_s_G_mean.num_errors)
            err_s_G_indiv.append(res_s_G_indiv.num_errors)
            err_h_G_indiv.append(res_h_G_indiv.num_errors)
            err_h_G_mean.append(res_h_G_mean.num_errors)
            err_s_K_mean.append(res_s_K_mean.num_errors)
            err_s_K_indiv.append(res_s_K_indiv.num_errors)
            err_h_K_indiv.append(res_h_K_indiv.num_errors)
            err_h_K_mean.append(res_h_K_mean.num_errors)
            err_s_KPS_mean.append(res_s_KPS_mean.num_errors)
            err_s_KPS_indiv.append(res_s_KPS_indiv.num_errors)
            err_h_KPS_indiv.append(res_h_KPS_indiv.num_errors)
            err_h_KPS_mean.append(res_h_KPS_mean.num_errors)

            err_h_G_mean_mean.append(res_h_G_mean_mean.num_errors)
            err_h_K_mean_mean.append(res_h_K_mean_mean.num_errors)
            err_h_KPS_mean_mean.append(res_h_KPS_mean_mean.num_errors)


        # Save the results
        json_Gaussian = {
            "decoding": "gaussian",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_G_mean,
                "soft_indiv": err_s_G_indiv,
                "hard_indiv": err_h_G_indiv,
                "hard_mean": err_h_G_mean,
                "hard_mean_mean": err_h_G_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_G_mean),
                "soft_indiv": np.mean(err_s_G_indiv),
                "hard_indiv": np.mean(err_h_G_indiv),
                "hard_mean": np.mean(err_h_G_mean),
                "hard_mean_mean": np.mean(err_h_G_mean_mean),
            },      
            "additional_info": {
                "calib_date": calib_date,
                "noise_list": noise_list,
                "calib_readout": readout,
                "p_soft_mean": p_soft_mean,
                "p_hard_mean": p_hard_mean,
                "p_softG_mean": pSoftG_mean,
            },    
        }

        json_KDE = {
            "decoding": "kde",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_K_mean,
                "soft_indiv": err_s_K_indiv,
                "hard_indiv": err_h_K_indiv,
                "hard_mean": err_h_K_mean,
                "hard_mean_mean": err_h_K_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_K_mean),
                "soft_indiv": np.mean(err_s_K_indiv),
                "hard_indiv": np.mean(err_h_K_indiv),
                "hard_mean": np.mean(err_h_K_mean),
                "hard_mean_mean": np.mean(err_h_K_mean_mean),
            },   
            "additional_info": {
                "rel_error": rel_error,
                "bandwidth_linspace": lin,
                "num_points_bandwidths": num_points,
                "pSoft_mean": pSoft_mean,
            },        
        }

        json_KDE_PS = {
            "decoding": "kde_PS",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_KPS_mean,
                "soft_indiv": err_s_KPS_indiv,
                "hard_indiv": err_h_KPS_indiv,
                "hard_mean": err_h_KPS_mean,
                "hard_mean_mean": err_h_KPS_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_KPS_mean),
                "soft_indiv": np.mean(err_s_KPS_indiv),
                "hard_indiv": np.mean(err_h_KPS_indiv),
                "hard_mean": np.mean(err_h_KPS_mean),
                "hard_mean_mean": np.mean(err_h_KPS_mean_mean),
                "pSoft_PS_mean": pSoft_PS_mean,
            },   
        }

        if not os.path.exists(file_name):
            data = {}
        else:
            with open(file_name, "r") as f:
                data = json.load(f)

        if job_id not in data.keys():
            data[job_id] = [{'totshots': NB_SHOTS}, json_Gaussian, json_KDE, json_KDE_PS]
        else:
            data[job_id].append(json_Gaussian)
            data[job_id].append(json_KDE)
            data[job_id].append(json_KDE_PS)
        
        with open(file_name, "w") as f:
            json.dump(data, f, indent=4)

20:06:26 Warning: Z0 decoding. Negative T2 error -0.12 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
20:06:26 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
20:06:26 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 104, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 09:05:05.295665+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 09:05:45.817173+00:00.
{0: {'p_hard': 0.004667259334518669, 'p_soft': 6.3500127000254e-05}, 1: {'p_hard': 0.012192024384048768, 'p_soft': 0.0043180

Retrieving jobs of 2024-03-23 08:28:22.511000+00:00 calibration: 100%|██████████| 1/1 [00:02<00:00,  2.65s/it]


Starting to get pSoft and countMat at 2024-03-31 20:08:40.869123
Starting to get pSoft_PS and countMat_PS at 2024-03-31 20:09:10.074628
Starting to get pSoftG at 2024-03-31 20:09:29.445810


100%|██████████| 12/12 [03:51<00:00, 19.31s/it]


# -----------------------------------------------------------

# -----------------------------------------------------------

# -----------------------------------------------------------

# -----------------------------------------------------------

In [ ]:
from Scratch import metadata_loader
import numpy as np
from tqdm import tqdm
from time import sleep

DEVICE = "ibm_sherbrooke"
LOGICAL = str(0)
XBASIS = True
ROUNDS = 100

state = "X" if XBASIS else "Z"
state += LOGICAL
print(f"state = {state}")


# Load the metadata
while True:
    try:
        md = metadata_loader(True, True)
        break
    except:
        sleep(5)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md[md["descr"] == 'subset RepCodes']
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
md = md[md["logical"] == LOGICAL]
md = md[md["xbasis"] == XBASIS]
md = md[md["rounds"] == ROUNDS]

# md = md[30:31]
print("shape:", md.shape)
# md[18:]
# md

state = Z0
shape: (40, 18)


# Group job ids by closest calibration date

In [ ]:
from datetime import datetime
from Scratch import find_closest_calib_jobs
import pandas as pd

DOUBLE_MSMT = False # to get also 03-16

jobs_by_calibration_date = {}
for index, row in md.iterrows():
    job_id = row['job_id']

    while True:
        try:
            _, _, calib_creation_date = find_closest_calib_jobs(tobecalib_job=job_id, verbose=False, double_msmt=DOUBLE_MSMT)
            break
        except:
            sleep(5)

    if calib_creation_date not in jobs_by_calibration_date.keys():
        jobs_by_calibration_date[calib_creation_date] = [job_id]
    else:
        jobs_by_calibration_date[calib_creation_date].append(job_id)

# Takes 15s

In [ ]:
print(jobs_by_calibration_date)
print()
print(f"num of calibrations: {len(jobs_by_calibration_date)}")
print(f"num of jobs per calibration: {([len(jobs) for jobs in jobs_by_calibration_date.values()])}")

{datetime.datetime(2024, 3, 23, 10, 49, 33, 412000, tzinfo=datetime.timezone.utc): ['cqzb7dds9z7g008dp66g', 'cqzb7cds9z7g008dp660', 'cqzb7bddvs8g008j66kg', 'cqzb7a5k5z70008j054g', 'cqzb7958gdp0008fvnj0', 'cqzb785s9z7g008dp650', 'cqzb774ktf3g00883jrg', 'cqzb764k5z70008j0540', 'cqzb754k5z70008j053g'], datetime.datetime(2024, 3, 23, 10, 48, 2, 487000, tzinfo=datetime.timezone.utc): ['cqzb744ktf3g00883jqg', 'cqzb734ktf3g00883jq0', 'cqzb724ktf3g00883jpg', 'cqzb71cczq6g0081h9yg', 'cqzb704ktf3g00883jp0', 'cqzb6yvczq6g0081h9y0', 'cqzb6xv8gdp0008fvngg', 'cqzb6wkktf3g00883jng', 'cqzb6vk8gdp0008fvng0', 'cqzb6tkk5z70008j0530'], datetime.datetime(2024, 3, 15, 22, 22, 45, 907000, tzinfo=datetime.timezone.utc): ['cqtcmchtxzj0008xz2dg', 'cqtcmb9txzj0008xz2d0', 'cqtcma988ev000811y6g', 'cqtcm9936d60008hyr20', 'cqtcm8988ev000811y60', 'cqtcm7088ev000811y5g', 'cqtcm6088ev000811y50', 'cqtcm50ch2mg008qvbjg', 'cqtcm409nfw0008hrvmg', 'cqtcm2r36d60008hyr1g'], datetime.datetime(2024, 3, 15, 22, 21, 14, 43000, tz

# Decode data

In [ ]:
d = md["distance"].values[0]

distances = np.arange(7, d+1, 4)
distances = distances[::-1]

distances

array([51, 47, 43, 39, 35, 31, 27, 23, 19, 15, 11,  7])

In [ ]:
import json
import os
import time

import pymatching
import stim

from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, get_repcode_IQ_map
from soft_info import RepetitionCodeStimCircuit, inv_qubit_mapping, gaussianIQConvertor
from soft_info import get_cols_to_keep, generate_subsets_with_center, get_subsample_layout
from Scratch import load_calibration_memory
import cpp_soft_info as csi

file_name  = f'./results_night1/{DEVICE}_{state}_{ROUNDS}.json'

HANDLE_OUTLIERS = True

rel_error = 1
_RESETS = False
nb_intervals = -1

# KDE BANDWIDTHS
lin = [0.6, 1.2, 1]
num_points = 20
# lin = [0.1, 0.7, 1]
# num_points = 7
bandwidths = np.linspace(lin[0], lin[1], lin[2])


for calib_date in jobs_by_calibration_date.keys():
    # get the noise dict of that date
    noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

    # get the KDE of that date

    while True:
        try:
            all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                        other_date=calib_date, post_process=True,
                                                                        double_msmt=False)
            break
        except:
            sleep(5)

    kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

    while True:
        try:
            all_memories_PS, gmm_dict_PS, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                            other_date=calib_date, post_process=True,
                                                                            double_msmt=True)
            break
        except:
            sleep(5)
    kde_dict_PS = csi.get_KDEs(all_memories_PS, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

    # Get the mean msmt errors
    p_soft_mean = 0
    p_hard_mean = 0
    for key, value in msmt_err_dict_PS.items():
        p_soft_mean += value['p_soft']
        p_hard_mean += value['p_hard']
    p_soft_mean /= len(msmt_err_dict_PS)
    p_hard_mean /= len(msmt_err_dict_PS)
    print(msmt_err_dict_PS, p_soft_mean, p_hard_mean)

    for i in range(127):
        print(f"qubit {i}: {kde_dict[i].bestBandwidth}", end=' ')
    print("\nPS", end=' ')
    for i in range(127):
        print(f"qubit {i}: {kde_dict_PS[i].bestBandwidth}", end=' ')

    # Retrieve the memories
    memories = []
    for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Retrieving jobs of {calib_date} calibration"):
        d = md[md["job_id"] == job_id]["distance"].values[0] 
        T = md[md["job_id"] == job_id]["rounds"].values[0] # Should be 10

        # Get the job
        job = provider.retrieve_job(job_id)
        memory = job.result().get_memory()
        memories.append(memory)

    # Stack the memories vertically
    big_memory = np.vstack(memories)
    NB_SHOTS = big_memory.shape[0]

    # Get the layout of the last job (same as previous)
    layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
    link_qubits = list(layout_des['link_qubit'].values())
    code_qubits = list(layout_des['code_qubit'].values())

    # Get the pSoft and countMat matrices
    big_layout = link_qubits + code_qubits
    inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))

    print(f"Starting to get pSoft and countMat at {datetime.now()}")
    pSoft, countMat = csi.iqConvertor(big_memory, inverted_q_map, kde_dict, rel_error, -1,
                                                handleOutliers = HANDLE_OUTLIERS)
    
    print(f"Starting to get pSoft_PS and countMat_PS at {datetime.now()}")
    pSoft_PS, countMat_PS = csi.iqConvertor(big_memory, inverted_q_map, kde_dict_PS, rel_error, -1,
                                                      handleOutliers = HANDLE_OUTLIERS)
    
    print(f"Starting to get pSoftG at {datetime.now()}")
    countMatG, pSoftG = gaussianIQConvertor(memory, inverted_q_map, gmm_dict)


    # Subsample decoding
    for D_NEW in tqdm(distances):
        subsets = generate_subsets_with_center(d, D_NEW)

        err_s_G_mean = []
        err_s_G_indiv = []
        err_h_G_indiv = []
        err_h_G_mean = []
        err_s_K_mean = []
        err_s_K_indiv = []
        err_h_K_indiv = []
        err_h_K_mean = []
        err_s_KPS_mean = []
        err_s_KPS_indiv = []
        err_h_KPS_indiv = []
        err_h_KPS_mean = []
        err_h_G_mean_mean = []
        err_h_K_mean_mean = []
        err_h_KPS_mean_mean = []
        for subset in (subsets):             
            cols_to_keep = get_cols_to_keep(subset, T, d)

            # Get the subset of pSoft and countMat
            pSoft_sub = pSoft[:, cols_to_keep]
            pSoft_PS_sub = pSoft_PS[:, cols_to_keep]
            pSoftG_sub = pSoftG[:, cols_to_keep]
            countMat_sub = countMat[:, cols_to_keep]
            countMat_PS_sub = countMat_PS[:, cols_to_keep]
            countMatG_sub = countMatG[:, cols_to_keep]

            # Means for hard decoding
            pSoft_mean = np.mean(pSoft_sub)
            pSoft_PS_mean = np.mean(pSoft_PS_sub)
            pSoftG_mean = np.mean(pSoftG_sub)

            # Get the layout
            layout = get_subsample_layout(subset, link_qubits, code_qubits)
            qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=T)

            # Get the noise avgs
            avgs = get_avgs_from_dict(noise_dict, layout)
            noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
            readout = avgs["readout"]
            noise_list += [(p_hard_mean+p_soft_mean), p_hard_mean, p_soft_mean]
            #[twog_err, sglg_err, t1_err, t2_err, readout_err, hard_err, soft_err]

            # Stim models
            subsampling = (D_NEW != d)

            code_indiv_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=layout,
                                                            msmt_err_dict=msmt_err_dict_PS)
            model_indiv_for_soft = code_indiv_for_soft.circuits[LOGICAL].detector_error_model()

            code_indiv_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=layout,
                                                            msmt_err_dict=msmt_err_dict_PS)
            model_indiv_for_hard = code_indiv_for_hard.circuits[LOGICAL].detector_error_model()

            code_mean_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_soft = code_mean_for_soft.circuits[LOGICAL].detector_error_model()

            code_mean_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_hard = code_mean_for_hard.circuits[LOGICAL].detector_error_model()
            

            # Decoding

            ### Gaussian
            res_s_G_mean = csi.decodeConvertorAll(model_mean_for_soft, countMatG_sub, pSoftG_sub, T,
                                                  int(LOGICAL), _RESETS, decode_hard=False)
            res_s_G_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_G_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_G_mean = csi.decodeConvertorAll(model_mean_for_hard, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoftG_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_G_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            ### KDE
            res_s_K_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_s_K_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_K_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_K_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoft_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                        subsampling=subsampling, no_fin_soft=False, layout=None,
                                                        msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_K_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
            
            ### KDE PS
            res_s_KPS_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_s_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_KPS_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoft_PS_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                        subsampling=subsampling, no_fin_soft=False, layout=None,
                                                        msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_KPS_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
            
            # Append the results
            err_s_G_mean.append(res_s_G_mean.num_errors)
            err_s_G_indiv.append(res_s_G_indiv.num_errors)
            err_h_G_indiv.append(res_h_G_indiv.num_errors)
            err_h_G_mean.append(res_h_G_mean.num_errors)
            err_s_K_mean.append(res_s_K_mean.num_errors)
            err_s_K_indiv.append(res_s_K_indiv.num_errors)
            err_h_K_indiv.append(res_h_K_indiv.num_errors)
            err_h_K_mean.append(res_h_K_mean.num_errors)
            err_s_KPS_mean.append(res_s_KPS_mean.num_errors)
            err_s_KPS_indiv.append(res_s_KPS_indiv.num_errors)
            err_h_KPS_indiv.append(res_h_KPS_indiv.num_errors)
            err_h_KPS_mean.append(res_h_KPS_mean.num_errors)

            err_h_G_mean_mean.append(res_h_G_mean_mean.num_errors)
            err_h_K_mean_mean.append(res_h_K_mean_mean.num_errors)
            err_h_KPS_mean_mean.append(res_h_KPS_mean_mean.num_errors)


        # Save the results
        json_Gaussian = {
            "decoding": "gaussian",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_G_mean,
                "soft_indiv": err_s_G_indiv,
                "hard_indiv": err_h_G_indiv,
                "hard_mean": err_h_G_mean,
                "hard_mean_mean": err_h_G_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_G_mean),
                "soft_indiv": np.mean(err_s_G_indiv),
                "hard_indiv": np.mean(err_h_G_indiv),
                "hard_mean": np.mean(err_h_G_mean),
                "hard_mean_mean": np.mean(err_h_G_mean_mean),
            },      
            "additional_info": {
                "calib_date": calib_date,
                "noise_list": noise_list,
                "calib_readout": readout,
                "p_soft_mean": p_soft_mean,
                "p_hard_mean": p_hard_mean,
                "p_softG_mean": pSoftG_mean,
            },    
        }

        json_KDE = {
            "decoding": "kde",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_K_mean,
                "soft_indiv": err_s_K_indiv,
                "hard_indiv": err_h_K_indiv,
                "hard_mean": err_h_K_mean,
                "hard_mean_mean": err_h_K_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_K_mean),
                "soft_indiv": np.mean(err_s_K_indiv),
                "hard_indiv": np.mean(err_h_K_indiv),
                "hard_mean": np.mean(err_h_K_mean),
                "hard_mean_mean": np.mean(err_h_K_mean_mean),
            },   
            "additional_info": {
                "rel_error": rel_error,
                "bandwidth_linspace": lin,
                "num_points_bandwidths": num_points,
                "pSoft_mean": pSoft_mean,
            },        
        }

        json_KDE_PS = {
            "decoding": "kde_PS",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_KPS_mean,
                "soft_indiv": err_s_KPS_indiv,
                "hard_indiv": err_h_KPS_indiv,
                "hard_mean": err_h_KPS_mean,
                "hard_mean_mean": err_h_KPS_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_KPS_mean),
                "soft_indiv": np.mean(err_s_KPS_indiv),
                "hard_indiv": np.mean(err_h_KPS_indiv),
                "hard_mean": np.mean(err_h_KPS_mean),
                "hard_mean_mean": np.mean(err_h_KPS_mean_mean),
                "pSoft_PS_mean": pSoft_PS_mean,
            },   
        }

        if not os.path.exists(file_name):
            data = {}
        else:
            with open(file_name, "r") as f:
                data = json.load(f)

        if job_id not in data.keys():
            data[job_id] = [{'totshots': NB_SHOTS}, json_Gaussian, json_KDE, json_KDE_PS]
        else:
            data[job_id].append(json_Gaussian)
            data[job_id].append(json_KDE)
            data[job_id].append(json_KDE_PS)
        
        with open(file_name, "w") as f:
            json.dump(data, f, indent=4)

20:06:26 Warning: Z0 decoding. Negative T2 error -0.12 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
20:06:26 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
20:06:26 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 104, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 09:05:05.295665+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 09:05:45.817173+00:00.
{0: {'p_hard': 0.004667259334518669, 'p_soft': 6.3500127000254e-05}, 1: {'p_hard': 0.012192024384048768, 'p_soft': 0.0043180

Retrieving jobs of 2024-03-23 08:28:22.511000+00:00 calibration: 100%|██████████| 1/1 [00:02<00:00,  2.65s/it]


Starting to get pSoft and countMat at 2024-03-31 20:08:40.869123
Starting to get pSoft_PS and countMat_PS at 2024-03-31 20:09:10.074628
Starting to get pSoftG at 2024-03-31 20:09:29.445810


100%|██████████| 12/12 [03:51<00:00, 19.31s/it]


# ----------------------------------------------------------

# ----------------------------------------------------------

In [ ]:
from Scratch import metadata_loader
import numpy as np
from tqdm import tqdm
from time import sleep

DEVICE = "ibm_sherbrooke"
LOGICAL = str(0)
XBASIS = True
ROUNDS = 10

state = "X" if XBASIS else "Z"
state += LOGICAL
print(f"state = {state}")


# Load the metadata
while True:
    try:
        md = metadata_loader(True, True)
        break
    except:
        sleep(5)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md[md["descr"] == 'subset RepCodes']
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
md = md[md["logical"] == LOGICAL]
md = md[md["xbasis"] == XBASIS]
md = md[md["rounds"] == ROUNDS]

# md = md[30:31]
print("shape:", md.shape)
# md[18:]
# md

state = X0
shape: (36, 18)


# Group job ids by closest calibration date

In [ ]:
from datetime import datetime
from Scratch import find_closest_calib_jobs
import pandas as pd

DOUBLE_MSMT = False # to get also 03-16

jobs_by_calibration_date = {}
for index, row in md.iterrows():
    job_id = row['job_id']

    while True:
        try:
            _, _, calib_creation_date = find_closest_calib_jobs(tobecalib_job=job_id, verbose=False, double_msmt=DOUBLE_MSMT)
            break
        except:
            sleep(5)

    if calib_creation_date not in jobs_by_calibration_date.keys():
        jobs_by_calibration_date[calib_creation_date] = [job_id]
    else:
        jobs_by_calibration_date[calib_creation_date].append(job_id)

# Takes 15s

In [ ]:
print(jobs_by_calibration_date)
print()
print(f"num of calibrations: {len(jobs_by_calibration_date)}")
print(f"num of jobs per calibration: {([len(jobs) for jobs in jobs_by_calibration_date.values()])}")

{datetime.datetime(2024, 3, 23, 10, 49, 33, 412000, tzinfo=datetime.timezone.utc): ['cqzb7dds9z7g008dp66g', 'cqzb7cds9z7g008dp660', 'cqzb7bddvs8g008j66kg', 'cqzb7a5k5z70008j054g', 'cqzb7958gdp0008fvnj0', 'cqzb785s9z7g008dp650', 'cqzb774ktf3g00883jrg', 'cqzb764k5z70008j0540', 'cqzb754k5z70008j053g'], datetime.datetime(2024, 3, 23, 10, 48, 2, 487000, tzinfo=datetime.timezone.utc): ['cqzb744ktf3g00883jqg', 'cqzb734ktf3g00883jq0', 'cqzb724ktf3g00883jpg', 'cqzb71cczq6g0081h9yg', 'cqzb704ktf3g00883jp0', 'cqzb6yvczq6g0081h9y0', 'cqzb6xv8gdp0008fvngg', 'cqzb6wkktf3g00883jng', 'cqzb6vk8gdp0008fvng0', 'cqzb6tkk5z70008j0530'], datetime.datetime(2024, 3, 15, 22, 22, 45, 907000, tzinfo=datetime.timezone.utc): ['cqtcmchtxzj0008xz2dg', 'cqtcmb9txzj0008xz2d0', 'cqtcma988ev000811y6g', 'cqtcm9936d60008hyr20', 'cqtcm8988ev000811y60', 'cqtcm7088ev000811y5g', 'cqtcm6088ev000811y50', 'cqtcm50ch2mg008qvbjg', 'cqtcm409nfw0008hrvmg', 'cqtcm2r36d60008hyr1g'], datetime.datetime(2024, 3, 15, 22, 21, 14, 43000, tz

# Decode data

In [ ]:
d = md["distance"].values[0]

distances = np.arange(7, d+1, 4)
distances = distances[::-1]

distances

array([51, 47, 43, 39, 35, 31, 27, 23, 19, 15, 11,  7])

In [ ]:
import json
import os
import time

import pymatching
import stim

from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, get_repcode_IQ_map
from soft_info import RepetitionCodeStimCircuit, inv_qubit_mapping, gaussianIQConvertor
from soft_info import get_cols_to_keep, generate_subsets_with_center, get_subsample_layout
from Scratch import load_calibration_memory
import cpp_soft_info as csi

file_name  = f'./results_night1/{DEVICE}_{state}_{ROUNDS}.json'

HANDLE_OUTLIERS = True

rel_error = 1
_RESETS = False
nb_intervals = -1

# KDE BANDWIDTHS
lin = [0.6, 1.2, 1]
num_points = 20
# lin = [0.1, 0.7, 1]
# num_points = 7
bandwidths = np.linspace(lin[0], lin[1], lin[2])


for calib_date in jobs_by_calibration_date.keys():
    # get the noise dict of that date
    noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

    # get the KDE of that date

    while True:
        try:
            all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                        other_date=calib_date, post_process=True,
                                                                        double_msmt=False)
            break
        except:
            sleep(5)

    kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

    while True:
        try:
            all_memories_PS, gmm_dict_PS, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                            other_date=calib_date, post_process=True,
                                                                            double_msmt=True)
            break
        except:
            sleep(5)
    kde_dict_PS = csi.get_KDEs(all_memories_PS, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

    # Get the mean msmt errors
    p_soft_mean = 0
    p_hard_mean = 0
    for key, value in msmt_err_dict_PS.items():
        p_soft_mean += value['p_soft']
        p_hard_mean += value['p_hard']
    p_soft_mean /= len(msmt_err_dict_PS)
    p_hard_mean /= len(msmt_err_dict_PS)
    print(msmt_err_dict_PS, p_soft_mean, p_hard_mean)

    for i in range(127):
        print(f"qubit {i}: {kde_dict[i].bestBandwidth}", end=' ')
    print("\nPS", end=' ')
    for i in range(127):
        print(f"qubit {i}: {kde_dict_PS[i].bestBandwidth}", end=' ')

    # Retrieve the memories
    memories = []
    for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Retrieving jobs of {calib_date} calibration"):
        d = md[md["job_id"] == job_id]["distance"].values[0] 
        T = md[md["job_id"] == job_id]["rounds"].values[0] # Should be 10

        # Get the job
        job = provider.retrieve_job(job_id)
        memory = job.result().get_memory()
        memories.append(memory)

    # Stack the memories vertically
    big_memory = np.vstack(memories)
    NB_SHOTS = big_memory.shape[0]

    # Get the layout of the last job (same as previous)
    layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
    link_qubits = list(layout_des['link_qubit'].values())
    code_qubits = list(layout_des['code_qubit'].values())

    # Get the pSoft and countMat matrices
    big_layout = link_qubits + code_qubits
    inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))

    print(f"Starting to get pSoft and countMat at {datetime.now()}")
    pSoft, countMat = csi.iqConvertor(big_memory, inverted_q_map, kde_dict, rel_error, -1,
                                                handleOutliers = HANDLE_OUTLIERS)
    
    print(f"Starting to get pSoft_PS and countMat_PS at {datetime.now()}")
    pSoft_PS, countMat_PS = csi.iqConvertor(big_memory, inverted_q_map, kde_dict_PS, rel_error, -1,
                                                      handleOutliers = HANDLE_OUTLIERS)
    
    print(f"Starting to get pSoftG at {datetime.now()}")
    countMatG, pSoftG = gaussianIQConvertor(memory, inverted_q_map, gmm_dict)


    # Subsample decoding
    for D_NEW in tqdm(distances):
        subsets = generate_subsets_with_center(d, D_NEW)

        err_s_G_mean = []
        err_s_G_indiv = []
        err_h_G_indiv = []
        err_h_G_mean = []
        err_s_K_mean = []
        err_s_K_indiv = []
        err_h_K_indiv = []
        err_h_K_mean = []
        err_s_KPS_mean = []
        err_s_KPS_indiv = []
        err_h_KPS_indiv = []
        err_h_KPS_mean = []
        err_h_G_mean_mean = []
        err_h_K_mean_mean = []
        err_h_KPS_mean_mean = []
        for subset in (subsets):             
            cols_to_keep = get_cols_to_keep(subset, T, d)

            # Get the subset of pSoft and countMat
            pSoft_sub = pSoft[:, cols_to_keep]
            pSoft_PS_sub = pSoft_PS[:, cols_to_keep]
            pSoftG_sub = pSoftG[:, cols_to_keep]
            countMat_sub = countMat[:, cols_to_keep]
            countMat_PS_sub = countMat_PS[:, cols_to_keep]
            countMatG_sub = countMatG[:, cols_to_keep]

            # Means for hard decoding
            pSoft_mean = np.mean(pSoft_sub)
            pSoft_PS_mean = np.mean(pSoft_PS_sub)
            pSoftG_mean = np.mean(pSoftG_sub)

            # Get the layout
            layout = get_subsample_layout(subset, link_qubits, code_qubits)
            qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=T)

            # Get the noise avgs
            avgs = get_avgs_from_dict(noise_dict, layout)
            noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
            readout = avgs["readout"]
            noise_list += [(p_hard_mean+p_soft_mean), p_hard_mean, p_soft_mean]
            #[twog_err, sglg_err, t1_err, t2_err, readout_err, hard_err, soft_err]

            # Stim models
            subsampling = (D_NEW != d)

            code_indiv_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=layout,
                                                            msmt_err_dict=msmt_err_dict_PS)
            model_indiv_for_soft = code_indiv_for_soft.circuits[LOGICAL].detector_error_model()

            code_indiv_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=layout,
                                                            msmt_err_dict=msmt_err_dict_PS)
            model_indiv_for_hard = code_indiv_for_hard.circuits[LOGICAL].detector_error_model()

            code_mean_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_soft = code_mean_for_soft.circuits[LOGICAL].detector_error_model()

            code_mean_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_hard = code_mean_for_hard.circuits[LOGICAL].detector_error_model()
            

            # Decoding

            ### Gaussian
            res_s_G_mean = csi.decodeConvertorAll(model_mean_for_soft, countMatG_sub, pSoftG_sub, T,
                                                  int(LOGICAL), _RESETS, decode_hard=False)
            res_s_G_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_G_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_G_mean = csi.decodeConvertorAll(model_mean_for_hard, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoftG_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_G_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            ### KDE
            res_s_K_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_s_K_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_K_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_K_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoft_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                        subsampling=subsampling, no_fin_soft=False, layout=None,
                                                        msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_K_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
            
            ### KDE PS
            res_s_KPS_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_s_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_KPS_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoft_PS_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                        subsampling=subsampling, no_fin_soft=False, layout=None,
                                                        msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_KPS_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
            
            # Append the results
            err_s_G_mean.append(res_s_G_mean.num_errors)
            err_s_G_indiv.append(res_s_G_indiv.num_errors)
            err_h_G_indiv.append(res_h_G_indiv.num_errors)
            err_h_G_mean.append(res_h_G_mean.num_errors)
            err_s_K_mean.append(res_s_K_mean.num_errors)
            err_s_K_indiv.append(res_s_K_indiv.num_errors)
            err_h_K_indiv.append(res_h_K_indiv.num_errors)
            err_h_K_mean.append(res_h_K_mean.num_errors)
            err_s_KPS_mean.append(res_s_KPS_mean.num_errors)
            err_s_KPS_indiv.append(res_s_KPS_indiv.num_errors)
            err_h_KPS_indiv.append(res_h_KPS_indiv.num_errors)
            err_h_KPS_mean.append(res_h_KPS_mean.num_errors)

            err_h_G_mean_mean.append(res_h_G_mean_mean.num_errors)
            err_h_K_mean_mean.append(res_h_K_mean_mean.num_errors)
            err_h_KPS_mean_mean.append(res_h_KPS_mean_mean.num_errors)


        # Save the results
        json_Gaussian = {
            "decoding": "gaussian",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_G_mean,
                "soft_indiv": err_s_G_indiv,
                "hard_indiv": err_h_G_indiv,
                "hard_mean": err_h_G_mean,
                "hard_mean_mean": err_h_G_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_G_mean),
                "soft_indiv": np.mean(err_s_G_indiv),
                "hard_indiv": np.mean(err_h_G_indiv),
                "hard_mean": np.mean(err_h_G_mean),
                "hard_mean_mean": np.mean(err_h_G_mean_mean),
            },      
            "additional_info": {
                "calib_date": calib_date,
                "noise_list": noise_list,
                "calib_readout": readout,
                "p_soft_mean": p_soft_mean,
                "p_hard_mean": p_hard_mean,
                "p_softG_mean": pSoftG_mean,
            },    
        }

        json_KDE = {
            "decoding": "kde",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_K_mean,
                "soft_indiv": err_s_K_indiv,
                "hard_indiv": err_h_K_indiv,
                "hard_mean": err_h_K_mean,
                "hard_mean_mean": err_h_K_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_K_mean),
                "soft_indiv": np.mean(err_s_K_indiv),
                "hard_indiv": np.mean(err_h_K_indiv),
                "hard_mean": np.mean(err_h_K_mean),
                "hard_mean_mean": np.mean(err_h_K_mean_mean),
            },   
            "additional_info": {
                "rel_error": rel_error,
                "bandwidth_linspace": lin,
                "num_points_bandwidths": num_points,
                "pSoft_mean": pSoft_mean,
            },        
        }

        json_KDE_PS = {
            "decoding": "kde_PS",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_KPS_mean,
                "soft_indiv": err_s_KPS_indiv,
                "hard_indiv": err_h_KPS_indiv,
                "hard_mean": err_h_KPS_mean,
                "hard_mean_mean": err_h_KPS_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_KPS_mean),
                "soft_indiv": np.mean(err_s_KPS_indiv),
                "hard_indiv": np.mean(err_h_KPS_indiv),
                "hard_mean": np.mean(err_h_KPS_mean),
                "hard_mean_mean": np.mean(err_h_KPS_mean_mean),
                "pSoft_PS_mean": pSoft_PS_mean,
            },   
        }

        if not os.path.exists(file_name):
            data = {}
        else:
            with open(file_name, "r") as f:
                data = json.load(f)

        if job_id not in data.keys():
            data[job_id] = [{'totshots': NB_SHOTS}, json_Gaussian, json_KDE, json_KDE_PS]
        else:
            data[job_id].append(json_Gaussian)
            data[job_id].append(json_KDE)
            data[job_id].append(json_KDE_PS)
        
        with open(file_name, "w") as f:
            json.dump(data, f, indent=4)

20:06:26 Warning: Z0 decoding. Negative T2 error -0.12 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
20:06:26 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
20:06:26 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 104, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 09:05:05.295665+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 09:05:45.817173+00:00.
{0: {'p_hard': 0.004667259334518669, 'p_soft': 6.3500127000254e-05}, 1: {'p_hard': 0.012192024384048768, 'p_soft': 0.0043180

Retrieving jobs of 2024-03-23 08:28:22.511000+00:00 calibration: 100%|██████████| 1/1 [00:02<00:00,  2.65s/it]


Starting to get pSoft and countMat at 2024-03-31 20:08:40.869123
Starting to get pSoft_PS and countMat_PS at 2024-03-31 20:09:10.074628
Starting to get pSoftG at 2024-03-31 20:09:29.445810


100%|██████████| 12/12 [03:51<00:00, 19.31s/it]


# ----------------------------------------------------------

# ----------------------------------------------------------

# ----------------------------------------------------------

# ----------------------------------------------------------

# ----------------------------------------------------------

In [ ]:
from Scratch import metadata_loader
import numpy as np
from tqdm import tqdm
from time import sleep

DEVICE = "ibm_sherbrooke"
LOGICAL = str(1)
XBASIS = True
ROUNDS = 10

state = "X" if XBASIS else "Z"
state += LOGICAL
print(f"state = {state}")


# Load the metadata
while True:
    try:
        md = metadata_loader(True, True)
        break
    except:
        sleep(5)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md[md["descr"] == 'subset RepCodes']
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
md = md[md["logical"] == LOGICAL]
md = md[md["xbasis"] == XBASIS]
md = md[md["rounds"] == ROUNDS]

# md = md[30:31]
print("shape:", md.shape)
# md[18:]
# md

state = X1


shape: (40, 18)


# Group job ids by closest calibration date

In [ ]:
from datetime import datetime
from Scratch import find_closest_calib_jobs
import pandas as pd

DOUBLE_MSMT = False # to get also 03-16

jobs_by_calibration_date = {}
for index, row in md.iterrows():
    job_id = row['job_id']

    while True:
        try:
            _, _, calib_creation_date = find_closest_calib_jobs(tobecalib_job=job_id, verbose=False, double_msmt=DOUBLE_MSMT)
            break
        except:
            sleep(5)

    if calib_creation_date not in jobs_by_calibration_date.keys():
        jobs_by_calibration_date[calib_creation_date] = [job_id]
    else:
        jobs_by_calibration_date[calib_creation_date].append(job_id)

# Takes 15s

In [ ]:
print(jobs_by_calibration_date)
print()
print(f"num of calibrations: {len(jobs_by_calibration_date)}")
print(f"num of jobs per calibration: {([len(jobs) for jobs in jobs_by_calibration_date.values()])}")

{datetime.datetime(2024, 3, 23, 10, 49, 33, 412000, tzinfo=datetime.timezone.utc): ['cqzb7dds9z7g008dp66g', 'cqzb7cds9z7g008dp660', 'cqzb7bddvs8g008j66kg', 'cqzb7a5k5z70008j054g', 'cqzb7958gdp0008fvnj0', 'cqzb785s9z7g008dp650', 'cqzb774ktf3g00883jrg', 'cqzb764k5z70008j0540', 'cqzb754k5z70008j053g'], datetime.datetime(2024, 3, 23, 10, 48, 2, 487000, tzinfo=datetime.timezone.utc): ['cqzb744ktf3g00883jqg', 'cqzb734ktf3g00883jq0', 'cqzb724ktf3g00883jpg', 'cqzb71cczq6g0081h9yg', 'cqzb704ktf3g00883jp0', 'cqzb6yvczq6g0081h9y0', 'cqzb6xv8gdp0008fvngg', 'cqzb6wkktf3g00883jng', 'cqzb6vk8gdp0008fvng0', 'cqzb6tkk5z70008j0530'], datetime.datetime(2024, 3, 15, 22, 22, 45, 907000, tzinfo=datetime.timezone.utc): ['cqtcmchtxzj0008xz2dg', 'cqtcmb9txzj0008xz2d0', 'cqtcma988ev000811y6g', 'cqtcm9936d60008hyr20', 'cqtcm8988ev000811y60', 'cqtcm7088ev000811y5g', 'cqtcm6088ev000811y50', 'cqtcm50ch2mg008qvbjg', 'cqtcm409nfw0008hrvmg', 'cqtcm2r36d60008hyr1g'], datetime.datetime(2024, 3, 15, 22, 21, 14, 43000, tz

# Decode data

In [ ]:
d = md["distance"].values[0]

distances = np.arange(7, d+1, 4)
distances = distances[::-1]

distances

array([51, 47, 43, 39, 35, 31, 27, 23, 19, 15, 11,  7])

In [ ]:
import json
import os
import time

import pymatching
import stim

from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, get_repcode_IQ_map
from soft_info import RepetitionCodeStimCircuit, inv_qubit_mapping, gaussianIQConvertor
from soft_info import get_cols_to_keep, generate_subsets_with_center, get_subsample_layout
from Scratch import load_calibration_memory
import cpp_soft_info as csi

file_name  = f'./results_night1/{DEVICE}_{state}_{ROUNDS}.json'

HANDLE_OUTLIERS = True

rel_error = 1
_RESETS = False
nb_intervals = -1

# KDE BANDWIDTHS
lin = [0.6, 1.2, 1]
num_points = 20
# lin = [0.1, 0.7, 1]
# num_points = 7
bandwidths = np.linspace(lin[0], lin[1], lin[2])


for calib_date in jobs_by_calibration_date.keys():
    # get the noise dict of that date
    noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

    # get the KDE of that date

    while True:
        try:
            all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                        other_date=calib_date, post_process=True,
                                                                        double_msmt=False)
            break
        except:
            sleep(5)

    kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

    while True:
        try:
            all_memories_PS, gmm_dict_PS, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                            other_date=calib_date, post_process=True,
                                                                            double_msmt=True)
            break
        except:
            sleep(5)
    kde_dict_PS = csi.get_KDEs(all_memories_PS, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

    # Get the mean msmt errors
    p_soft_mean = 0
    p_hard_mean = 0
    for key, value in msmt_err_dict_PS.items():
        p_soft_mean += value['p_soft']
        p_hard_mean += value['p_hard']
    p_soft_mean /= len(msmt_err_dict_PS)
    p_hard_mean /= len(msmt_err_dict_PS)
    print(msmt_err_dict_PS, p_soft_mean, p_hard_mean)

    for i in range(127):
        print(f"qubit {i}: {kde_dict[i].bestBandwidth}", end=' ')
    print("\nPS", end=' ')
    for i in range(127):
        print(f"qubit {i}: {kde_dict_PS[i].bestBandwidth}", end=' ')

    # Retrieve the memories
    memories = []
    for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Retrieving jobs of {calib_date} calibration"):
        d = md[md["job_id"] == job_id]["distance"].values[0] 
        T = md[md["job_id"] == job_id]["rounds"].values[0] # Should be 10

        # Get the job
        job = provider.retrieve_job(job_id)
        memory = job.result().get_memory()
        memories.append(memory)

    # Stack the memories vertically
    big_memory = np.vstack(memories)
    NB_SHOTS = big_memory.shape[0]

    # Get the layout of the last job (same as previous)
    layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
    link_qubits = list(layout_des['link_qubit'].values())
    code_qubits = list(layout_des['code_qubit'].values())

    # Get the pSoft and countMat matrices
    big_layout = link_qubits + code_qubits
    inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))

    print(f"Starting to get pSoft and countMat at {datetime.now()}")
    pSoft, countMat = csi.iqConvertor(big_memory, inverted_q_map, kde_dict, rel_error, -1,
                                                handleOutliers = HANDLE_OUTLIERS)
    
    print(f"Starting to get pSoft_PS and countMat_PS at {datetime.now()}")
    pSoft_PS, countMat_PS = csi.iqConvertor(big_memory, inverted_q_map, kde_dict_PS, rel_error, -1,
                                                      handleOutliers = HANDLE_OUTLIERS)
    
    print(f"Starting to get pSoftG at {datetime.now()}")
    countMatG, pSoftG = gaussianIQConvertor(memory, inverted_q_map, gmm_dict)


    # Subsample decoding
    for D_NEW in tqdm(distances):
        subsets = generate_subsets_with_center(d, D_NEW)

        err_s_G_mean = []
        err_s_G_indiv = []
        err_h_G_indiv = []
        err_h_G_mean = []
        err_s_K_mean = []
        err_s_K_indiv = []
        err_h_K_indiv = []
        err_h_K_mean = []
        err_s_KPS_mean = []
        err_s_KPS_indiv = []
        err_h_KPS_indiv = []
        err_h_KPS_mean = []
        err_h_G_mean_mean = []
        err_h_K_mean_mean = []
        err_h_KPS_mean_mean = []
        for subset in (subsets):             
            cols_to_keep = get_cols_to_keep(subset, T, d)

            # Get the subset of pSoft and countMat
            pSoft_sub = pSoft[:, cols_to_keep]
            pSoft_PS_sub = pSoft_PS[:, cols_to_keep]
            pSoftG_sub = pSoftG[:, cols_to_keep]
            countMat_sub = countMat[:, cols_to_keep]
            countMat_PS_sub = countMat_PS[:, cols_to_keep]
            countMatG_sub = countMatG[:, cols_to_keep]

            # Means for hard decoding
            pSoft_mean = np.mean(pSoft_sub)
            pSoft_PS_mean = np.mean(pSoft_PS_sub)
            pSoftG_mean = np.mean(pSoftG_sub)

            # Get the layout
            layout = get_subsample_layout(subset, link_qubits, code_qubits)
            qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=T)

            # Get the noise avgs
            avgs = get_avgs_from_dict(noise_dict, layout)
            noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
            readout = avgs["readout"]
            noise_list += [(p_hard_mean+p_soft_mean), p_hard_mean, p_soft_mean]
            #[twog_err, sglg_err, t1_err, t2_err, readout_err, hard_err, soft_err]

            # Stim models
            subsampling = (D_NEW != d)

            code_indiv_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=layout,
                                                            msmt_err_dict=msmt_err_dict_PS)
            model_indiv_for_soft = code_indiv_for_soft.circuits[LOGICAL].detector_error_model()

            code_indiv_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=layout,
                                                            msmt_err_dict=msmt_err_dict_PS)
            model_indiv_for_hard = code_indiv_for_hard.circuits[LOGICAL].detector_error_model()

            code_mean_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_soft = code_mean_for_soft.circuits[LOGICAL].detector_error_model()

            code_mean_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_hard = code_mean_for_hard.circuits[LOGICAL].detector_error_model()
            

            # Decoding

            ### Gaussian
            res_s_G_mean = csi.decodeConvertorAll(model_mean_for_soft, countMatG_sub, pSoftG_sub, T,
                                                  int(LOGICAL), _RESETS, decode_hard=False)
            res_s_G_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_G_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_G_mean = csi.decodeConvertorAll(model_mean_for_hard, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoftG_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_G_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            ### KDE
            res_s_K_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_s_K_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_K_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_K_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoft_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                        subsampling=subsampling, no_fin_soft=False, layout=None,
                                                        msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_K_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
            
            ### KDE PS
            res_s_KPS_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_s_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_KPS_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoft_PS_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                        subsampling=subsampling, no_fin_soft=False, layout=None,
                                                        msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_KPS_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
            
            # Append the results
            err_s_G_mean.append(res_s_G_mean.num_errors)
            err_s_G_indiv.append(res_s_G_indiv.num_errors)
            err_h_G_indiv.append(res_h_G_indiv.num_errors)
            err_h_G_mean.append(res_h_G_mean.num_errors)
            err_s_K_mean.append(res_s_K_mean.num_errors)
            err_s_K_indiv.append(res_s_K_indiv.num_errors)
            err_h_K_indiv.append(res_h_K_indiv.num_errors)
            err_h_K_mean.append(res_h_K_mean.num_errors)
            err_s_KPS_mean.append(res_s_KPS_mean.num_errors)
            err_s_KPS_indiv.append(res_s_KPS_indiv.num_errors)
            err_h_KPS_indiv.append(res_h_KPS_indiv.num_errors)
            err_h_KPS_mean.append(res_h_KPS_mean.num_errors)

            err_h_G_mean_mean.append(res_h_G_mean_mean.num_errors)
            err_h_K_mean_mean.append(res_h_K_mean_mean.num_errors)
            err_h_KPS_mean_mean.append(res_h_KPS_mean_mean.num_errors)


        # Save the results
        json_Gaussian = {
            "decoding": "gaussian",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_G_mean,
                "soft_indiv": err_s_G_indiv,
                "hard_indiv": err_h_G_indiv,
                "hard_mean": err_h_G_mean,
                "hard_mean_mean": err_h_G_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_G_mean),
                "soft_indiv": np.mean(err_s_G_indiv),
                "hard_indiv": np.mean(err_h_G_indiv),
                "hard_mean": np.mean(err_h_G_mean),
                "hard_mean_mean": np.mean(err_h_G_mean_mean),
            },      
            "additional_info": {
                "calib_date": calib_date,
                "noise_list": noise_list,
                "calib_readout": readout,
                "p_soft_mean": p_soft_mean,
                "p_hard_mean": p_hard_mean,
                "p_softG_mean": pSoftG_mean,
            },    
        }

        json_KDE = {
            "decoding": "kde",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_K_mean,
                "soft_indiv": err_s_K_indiv,
                "hard_indiv": err_h_K_indiv,
                "hard_mean": err_h_K_mean,
                "hard_mean_mean": err_h_K_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_K_mean),
                "soft_indiv": np.mean(err_s_K_indiv),
                "hard_indiv": np.mean(err_h_K_indiv),
                "hard_mean": np.mean(err_h_K_mean),
                "hard_mean_mean": np.mean(err_h_K_mean_mean),
            },   
            "additional_info": {
                "rel_error": rel_error,
                "bandwidth_linspace": lin,
                "num_points_bandwidths": num_points,
                "pSoft_mean": pSoft_mean,
            },        
        }

        json_KDE_PS = {
            "decoding": "kde_PS",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_KPS_mean,
                "soft_indiv": err_s_KPS_indiv,
                "hard_indiv": err_h_KPS_indiv,
                "hard_mean": err_h_KPS_mean,
                "hard_mean_mean": err_h_KPS_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_KPS_mean),
                "soft_indiv": np.mean(err_s_KPS_indiv),
                "hard_indiv": np.mean(err_h_KPS_indiv),
                "hard_mean": np.mean(err_h_KPS_mean),
                "hard_mean_mean": np.mean(err_h_KPS_mean_mean),
                "pSoft_PS_mean": pSoft_PS_mean,
            },   
        }

        if not os.path.exists(file_name):
            data = {}
        else:
            with open(file_name, "r") as f:
                data = json.load(f)

        if job_id not in data.keys():
            data[job_id] = [{'totshots': NB_SHOTS}, json_Gaussian, json_KDE, json_KDE_PS]
        else:
            data[job_id].append(json_Gaussian)
            data[job_id].append(json_KDE)
            data[job_id].append(json_KDE_PS)
        
        with open(file_name, "w") as f:
            json.dump(data, f, indent=4)

20:06:26 Warning: Z0 decoding. Negative T2 error -0.12 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
20:06:26 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
20:06:26 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 104, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 09:05:05.295665+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 09:05:45.817173+00:00.
{0: {'p_hard': 0.004667259334518669, 'p_soft': 6.3500127000254e-05}, 1: {'p_hard': 0.012192024384048768, 'p_soft': 0.0043180

Retrieving jobs of 2024-03-23 08:28:22.511000+00:00 calibration: 100%|██████████| 1/1 [00:02<00:00,  2.65s/it]


Starting to get pSoft and countMat at 2024-03-31 20:08:40.869123
Starting to get pSoft_PS and countMat_PS at 2024-03-31 20:09:10.074628
Starting to get pSoftG at 2024-03-31 20:09:29.445810


100%|██████████| 12/12 [03:51<00:00, 19.31s/it]


# ----------------------------------------------------------

# ----------------------------------------------------------

# ----------------------------------------------------------

# ----------------------------------------------------------

# ----------------------------------------------------------

In [ ]:
from Scratch import metadata_loader
import numpy as np
from tqdm import tqdm
from time import sleep

DEVICE = "ibm_sherbrooke"
LOGICAL = str(0)
XBASIS = False
ROUNDS = 10

state = "X" if XBASIS else "Z"
state += LOGICAL
print(f"state = {state}")


# Load the metadata
while True:
    try:
        md = metadata_loader(True, True)
        break
    except:
        sleep(5)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md[md["descr"] == 'subset RepCodes']
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
md = md[md["logical"] == LOGICAL]
md = md[md["xbasis"] == XBASIS]
md = md[md["rounds"] == ROUNDS]

# md = md[30:31]
print("shape:", md.shape)
# md[18:]
# md

state = Z0


shape: (40, 18)


# Group job ids by closest calibration date

In [ ]:
from datetime import datetime
from Scratch import find_closest_calib_jobs
import pandas as pd

DOUBLE_MSMT = False # to get also 03-16

jobs_by_calibration_date = {}
for index, row in md.iterrows():
    job_id = row['job_id']

    while True:
        try:
            _, _, calib_creation_date = find_closest_calib_jobs(tobecalib_job=job_id, verbose=False, double_msmt=DOUBLE_MSMT)
            break
        except:
            sleep(5)

    if calib_creation_date not in jobs_by_calibration_date.keys():
        jobs_by_calibration_date[calib_creation_date] = [job_id]
    else:
        jobs_by_calibration_date[calib_creation_date].append(job_id)

# Takes 15s

In [ ]:
print(jobs_by_calibration_date)
print()
print(f"num of calibrations: {len(jobs_by_calibration_date)}")
print(f"num of jobs per calibration: {([len(jobs) for jobs in jobs_by_calibration_date.values()])}")

{datetime.datetime(2024, 3, 23, 10, 49, 33, 412000, tzinfo=datetime.timezone.utc): ['cqzb7dds9z7g008dp66g', 'cqzb7cds9z7g008dp660', 'cqzb7bddvs8g008j66kg', 'cqzb7a5k5z70008j054g', 'cqzb7958gdp0008fvnj0', 'cqzb785s9z7g008dp650', 'cqzb774ktf3g00883jrg', 'cqzb764k5z70008j0540', 'cqzb754k5z70008j053g'], datetime.datetime(2024, 3, 23, 10, 48, 2, 487000, tzinfo=datetime.timezone.utc): ['cqzb744ktf3g00883jqg', 'cqzb734ktf3g00883jq0', 'cqzb724ktf3g00883jpg', 'cqzb71cczq6g0081h9yg', 'cqzb704ktf3g00883jp0', 'cqzb6yvczq6g0081h9y0', 'cqzb6xv8gdp0008fvngg', 'cqzb6wkktf3g00883jng', 'cqzb6vk8gdp0008fvng0', 'cqzb6tkk5z70008j0530'], datetime.datetime(2024, 3, 15, 22, 22, 45, 907000, tzinfo=datetime.timezone.utc): ['cqtcmchtxzj0008xz2dg', 'cqtcmb9txzj0008xz2d0', 'cqtcma988ev000811y6g', 'cqtcm9936d60008hyr20', 'cqtcm8988ev000811y60', 'cqtcm7088ev000811y5g', 'cqtcm6088ev000811y50', 'cqtcm50ch2mg008qvbjg', 'cqtcm409nfw0008hrvmg', 'cqtcm2r36d60008hyr1g'], datetime.datetime(2024, 3, 15, 22, 21, 14, 43000, tz

# Decode data

In [ ]:
d = md["distance"].values[0]

distances = np.arange(7, d+1, 4)
distances = distances[::-1]

distances

array([51, 47, 43, 39, 35, 31, 27, 23, 19, 15, 11,  7])

In [ ]:
import json
import os
import time

import pymatching
import stim

from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, get_repcode_IQ_map
from soft_info import RepetitionCodeStimCircuit, inv_qubit_mapping, gaussianIQConvertor
from soft_info import get_cols_to_keep, generate_subsets_with_center, get_subsample_layout
from Scratch import load_calibration_memory
import cpp_soft_info as csi

file_name  = f'./results_night1/{DEVICE}_{state}_{ROUNDS}.json'

HANDLE_OUTLIERS = True

rel_error = 1
_RESETS = False
nb_intervals = -1

# KDE BANDWIDTHS
lin = [0.6, 1.2, 1]
num_points = 20
# lin = [0.1, 0.7, 1]
# num_points = 7
bandwidths = np.linspace(lin[0], lin[1], lin[2])


for calib_date in jobs_by_calibration_date.keys():
    # get the noise dict of that date
    noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

    # get the KDE of that date

    while True:
        try:
            all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                        other_date=calib_date, post_process=True,
                                                                        double_msmt=False)
            break
        except:
            sleep(5)

    kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

    while True:
        try:
            all_memories_PS, gmm_dict_PS, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                            other_date=calib_date, post_process=True,
                                                                            double_msmt=True)
            break
        except:
            sleep(5)
    kde_dict_PS = csi.get_KDEs(all_memories_PS, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

    # Get the mean msmt errors
    p_soft_mean = 0
    p_hard_mean = 0
    for key, value in msmt_err_dict_PS.items():
        p_soft_mean += value['p_soft']
        p_hard_mean += value['p_hard']
    p_soft_mean /= len(msmt_err_dict_PS)
    p_hard_mean /= len(msmt_err_dict_PS)
    print(msmt_err_dict_PS, p_soft_mean, p_hard_mean)

    for i in range(127):
        print(f"qubit {i}: {kde_dict[i].bestBandwidth}", end=' ')
    print("\nPS", end=' ')
    for i in range(127):
        print(f"qubit {i}: {kde_dict_PS[i].bestBandwidth}", end=' ')

    # Retrieve the memories
    memories = []
    for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Retrieving jobs of {calib_date} calibration"):
        d = md[md["job_id"] == job_id]["distance"].values[0] 
        T = md[md["job_id"] == job_id]["rounds"].values[0] # Should be 10

        # Get the job
        job = provider.retrieve_job(job_id)
        memory = job.result().get_memory()
        memories.append(memory)

    # Stack the memories vertically
    big_memory = np.vstack(memories)
    NB_SHOTS = big_memory.shape[0]

    # Get the layout of the last job (same as previous)
    layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
    link_qubits = list(layout_des['link_qubit'].values())
    code_qubits = list(layout_des['code_qubit'].values())

    # Get the pSoft and countMat matrices
    big_layout = link_qubits + code_qubits
    inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))

    print(f"Starting to get pSoft and countMat at {datetime.now()}")
    pSoft, countMat = csi.iqConvertor(big_memory, inverted_q_map, kde_dict, rel_error, -1,
                                                handleOutliers = HANDLE_OUTLIERS)
    
    print(f"Starting to get pSoft_PS and countMat_PS at {datetime.now()}")
    pSoft_PS, countMat_PS = csi.iqConvertor(big_memory, inverted_q_map, kde_dict_PS, rel_error, -1,
                                                      handleOutliers = HANDLE_OUTLIERS)
    
    print(f"Starting to get pSoftG at {datetime.now()}")
    countMatG, pSoftG = gaussianIQConvertor(memory, inverted_q_map, gmm_dict)


    # Subsample decoding
    for D_NEW in tqdm(distances):
        subsets = generate_subsets_with_center(d, D_NEW)

        err_s_G_mean = []
        err_s_G_indiv = []
        err_h_G_indiv = []
        err_h_G_mean = []
        err_s_K_mean = []
        err_s_K_indiv = []
        err_h_K_indiv = []
        err_h_K_mean = []
        err_s_KPS_mean = []
        err_s_KPS_indiv = []
        err_h_KPS_indiv = []
        err_h_KPS_mean = []
        err_h_G_mean_mean = []
        err_h_K_mean_mean = []
        err_h_KPS_mean_mean = []
        for subset in (subsets):             
            cols_to_keep = get_cols_to_keep(subset, T, d)

            # Get the subset of pSoft and countMat
            pSoft_sub = pSoft[:, cols_to_keep]
            pSoft_PS_sub = pSoft_PS[:, cols_to_keep]
            pSoftG_sub = pSoftG[:, cols_to_keep]
            countMat_sub = countMat[:, cols_to_keep]
            countMat_PS_sub = countMat_PS[:, cols_to_keep]
            countMatG_sub = countMatG[:, cols_to_keep]

            # Means for hard decoding
            pSoft_mean = np.mean(pSoft_sub)
            pSoft_PS_mean = np.mean(pSoft_PS_sub)
            pSoftG_mean = np.mean(pSoftG_sub)

            # Get the layout
            layout = get_subsample_layout(subset, link_qubits, code_qubits)
            qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=T)

            # Get the noise avgs
            avgs = get_avgs_from_dict(noise_dict, layout)
            noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
            readout = avgs["readout"]
            noise_list += [(p_hard_mean+p_soft_mean), p_hard_mean, p_soft_mean]
            #[twog_err, sglg_err, t1_err, t2_err, readout_err, hard_err, soft_err]

            # Stim models
            subsampling = (D_NEW != d)

            code_indiv_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=layout,
                                                            msmt_err_dict=msmt_err_dict_PS)
            model_indiv_for_soft = code_indiv_for_soft.circuits[LOGICAL].detector_error_model()

            code_indiv_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=layout,
                                                            msmt_err_dict=msmt_err_dict_PS)
            model_indiv_for_hard = code_indiv_for_hard.circuits[LOGICAL].detector_error_model()

            code_mean_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_soft = code_mean_for_soft.circuits[LOGICAL].detector_error_model()

            code_mean_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_hard = code_mean_for_hard.circuits[LOGICAL].detector_error_model()
            

            # Decoding

            ### Gaussian
            res_s_G_mean = csi.decodeConvertorAll(model_mean_for_soft, countMatG_sub, pSoftG_sub, T,
                                                  int(LOGICAL), _RESETS, decode_hard=False)
            res_s_G_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_G_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_G_mean = csi.decodeConvertorAll(model_mean_for_hard, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoftG_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_G_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMatG_sub, pSoftG_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            ### KDE
            res_s_K_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_s_K_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_K_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_K_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_sub, pSoft_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoft_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                        subsampling=subsampling, no_fin_soft=False, layout=None,
                                                        msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_K_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
            
            ### KDE PS
            res_s_KPS_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_s_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=False)
            res_h_KPS_indiv = csi.decodeConvertorAll(model_indiv_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            res_h_KPS_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                    int(LOGICAL), _RESETS, decode_hard=True)
            
            noise_list[-1] = pSoft_PS_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                        subsampling=subsampling, no_fin_soft=False, layout=None,
                                                        msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()
            res_h_KPS_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
            
            # Append the results
            err_s_G_mean.append(res_s_G_mean.num_errors)
            err_s_G_indiv.append(res_s_G_indiv.num_errors)
            err_h_G_indiv.append(res_h_G_indiv.num_errors)
            err_h_G_mean.append(res_h_G_mean.num_errors)
            err_s_K_mean.append(res_s_K_mean.num_errors)
            err_s_K_indiv.append(res_s_K_indiv.num_errors)
            err_h_K_indiv.append(res_h_K_indiv.num_errors)
            err_h_K_mean.append(res_h_K_mean.num_errors)
            err_s_KPS_mean.append(res_s_KPS_mean.num_errors)
            err_s_KPS_indiv.append(res_s_KPS_indiv.num_errors)
            err_h_KPS_indiv.append(res_h_KPS_indiv.num_errors)
            err_h_KPS_mean.append(res_h_KPS_mean.num_errors)

            err_h_G_mean_mean.append(res_h_G_mean_mean.num_errors)
            err_h_K_mean_mean.append(res_h_K_mean_mean.num_errors)
            err_h_KPS_mean_mean.append(res_h_KPS_mean_mean.num_errors)


        # Save the results
        json_Gaussian = {
            "decoding": "gaussian",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_G_mean,
                "soft_indiv": err_s_G_indiv,
                "hard_indiv": err_h_G_indiv,
                "hard_mean": err_h_G_mean,
                "hard_mean_mean": err_h_G_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_G_mean),
                "soft_indiv": np.mean(err_s_G_indiv),
                "hard_indiv": np.mean(err_h_G_indiv),
                "hard_mean": np.mean(err_h_G_mean),
                "hard_mean_mean": np.mean(err_h_G_mean_mean),
            },      
            "additional_info": {
                "calib_date": calib_date,
                "noise_list": noise_list,
                "calib_readout": readout,
                "p_soft_mean": p_soft_mean,
                "p_hard_mean": p_hard_mean,
                "p_softG_mean": pSoftG_mean,
            },    
        }

        json_KDE = {
            "decoding": "kde",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_K_mean,
                "soft_indiv": err_s_K_indiv,
                "hard_indiv": err_h_K_indiv,
                "hard_mean": err_h_K_mean,
                "hard_mean_mean": err_h_K_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_K_mean),
                "soft_indiv": np.mean(err_s_K_indiv),
                "hard_indiv": np.mean(err_h_K_indiv),
                "hard_mean": np.mean(err_h_K_mean),
                "hard_mean_mean": np.mean(err_h_K_mean_mean),
            },   
            "additional_info": {
                "rel_error": rel_error,
                "bandwidth_linspace": lin,
                "num_points_bandwidths": num_points,
                "pSoft_mean": pSoft_mean,
            },        
        }

        json_KDE_PS = {
            "decoding": "kde_PS",
            "d_new": str(D_NEW),
            "error_list_dict": {
                "soft_mean": err_s_KPS_mean,
                "soft_indiv": err_s_KPS_indiv,
                "hard_indiv": err_h_KPS_indiv,
                "hard_mean": err_h_KPS_mean,
                "hard_mean_mean": err_h_KPS_mean_mean,
            },
            "error_mean_dict": {
                "soft_mean": np.mean(err_s_KPS_mean),
                "soft_indiv": np.mean(err_s_KPS_indiv),
                "hard_indiv": np.mean(err_h_KPS_indiv),
                "hard_mean": np.mean(err_h_KPS_mean),
                "hard_mean_mean": np.mean(err_h_KPS_mean_mean),
                "pSoft_PS_mean": pSoft_PS_mean,
            },   
        }

        if not os.path.exists(file_name):
            data = {}
        else:
            with open(file_name, "r") as f:
                data = json.load(f)

        if job_id not in data.keys():
            data[job_id] = [{'totshots': NB_SHOTS}, json_Gaussian, json_KDE, json_KDE_PS]
        else:
            data[job_id].append(json_Gaussian)
            data[job_id].append(json_KDE)
            data[job_id].append(json_KDE_PS)
        
        with open(file_name, "w") as f:
            json.dump(data, f, indent=4)

20:06:26 Warning: Z0 decoding. Negative T2 error -0.12 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
20:06:26 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
20:06:26 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 104, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 09:05:05.295665+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 09:05:45.817173+00:00.
{0: {'p_hard': 0.004667259334518669, 'p_soft': 6.3500127000254e-05}, 1: {'p_hard': 0.012192024384048768, 'p_soft': 0.0043180

Retrieving jobs of 2024-03-23 08:28:22.511000+00:00 calibration: 100%|██████████| 1/1 [00:02<00:00,  2.65s/it]


Starting to get pSoft and countMat at 2024-03-31 20:08:40.869123
Starting to get pSoft_PS and countMat_PS at 2024-03-31 20:09:10.074628
Starting to get pSoftG at 2024-03-31 20:09:29.445810


100%|██████████| 12/12 [03:51<00:00, 19.31s/it]
